In [3]:
import pandas as pd

# Specify the path to the JSON file
file_path = './dataset/train.json'

# Read the JSON file into a DataFrame
df = pd.read_json(file_path)

# Preview the first few rows of the DataFrame
df.head()


,2k7915,1k3845,1k8446,14k940,12k646,1k2794,2k1417,2k6436,11k434,2k4746,...,17k884,1k5329,17k734,1k1731,14k612,16k076,2k7441,2k2481,16k288,2k8444
title,Why is the NW map so small?,Any updates in regards to the Flame War?,Hey,Any tips for final exams?,My orthodontist just said I can't have nuts be...,[QUESTION] IS THE PRS SE CUSTOM 24 A GOOD QUAL...,Tweek and Craig,Hey has anybody been following this story with...,Is the best way to record VR to capture the pr...,Hossin full...,...,F4A (f19) control my Nora while I try to sleep?,My gf cheated and I'm not mad,r/BabyJevilCase Lounge,How to prepare for anal?,Running Epic Games from External Hard Drive,[Discussion] Can we appreciate the people in t...,Guys opinion?,What's the weirdest thing you've ever eaten?,Finishing a tower.,Monday morning m'thread
body,When did this change? It's not been fun spawni...,Just out of curiosity. I'm only wondering what...,"Im not phased by anything, love you all and I'...",I am a first year student in Bachelor of Scien...,What do I do I want to keep my nuts,Hey! I want to get this guitar but Im a bit wo...,Are Tweek and Craig actually in a relationship...,You fucking faggots.,When you're playing VR a preview of what you ...,... and others maps closed. Nice. This get old.,...,I'm feeling a bit naughty tonight sooooo,So my gf was counseling at a camp and I get a ...,A place for members of r/BabyJevilCase to chat...,So i want to try it out with my bf. How to pre...,Running out of space on my laptop and tried to...,Nothing worse or uneasing than completing a tr...,Hey guys.. so I sleep with my bf almost every ...,I once had a kangaroo burger from an Australia...,"So basically, I am making a game similar to JT...",Boo Monday is here. Good weekend? What did yo...
comment,Maybe they need to keep the map large to start...,Shut the fuck up freeloading asshat,MORE WIGGER SHIT TO DECODE,"For Calc2, do past exams \* 6, remember to exp...",just eat em and be careful it's fine,PRS SE's are probably the most consistent for ...,Things are starting to look good for our homos...,Nah man if I had I would tell you. Did hear th...,"As far as I know, yes that is the best way. Ho...",Only pussies QQ about Hossin.,...,"I'm up for that, dm me",Was he a NlGGER! Those Black animals are breed...,I just wouldn't play,Fast a little bit before. Go to the bathroom....,"It's not really a good idea to do that, but is...",Who doesn't say ty? Rude,He is obviously an empty shell. Most males wh...,"I just tried a ""chumbo"" in Andalusia, it's bas...",make sure you allow practice time for free,"Fuck off Monday. Got the dentist on Tuesday, a..."
L1,0,3,3,0,0,0,0,2,0,3,...,3,1,0,1,0,0,3,0,0,1
L2,0,1,1,0,0,0,0,1,0,1,...,1,1,0,1,0,0,1,0,0,1


Testing the performance of using the standard BERT model for detecting hate-speech in language. 

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import torch

# Specify the path to the JSON file
file_path = './dataset/train.json'

# Read the JSON file into a DataFrame
df = pd.read_json(file_path)

# Extract the text and labels from the DataFrame
texts = df['text'].tolist()
labels = df['label'].tolist()

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text and convert it to input features
input_ids = []
attention_masks = []
for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Create a DataLoader for batching the input features
batch_size = 16
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    progress_bar = tqdm(dataloader, desc='Epoch {}/{}'.format(epoch+1, epochs), leave=False)
    for batch in progress_bar:
        input_ids_batch, attention_masks_batch, labels_batch = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch, labels=labels_batch)
        loss = outputs.loss
        logits = outputs.logits
        epoch_loss += loss.item()
        _, predictions = torch.max(logits, dim=1)
        correct_predictions = torch.sum(predictions == labels_batch)
        epoch_accuracy += correct_predictions.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        progress_bar.set_postfix({'loss': loss.item(), 'accuracy': correct_predictions.item()/batch_size})
    epoch_loss /= len(dataloader)
    epoch_accuracy /= len(dataset)
    print('Epoch {}/{}: Loss {:.4f}, Accuracy {:.4f}'.format(epoch+1, epochs, epoch_loss, epoch_accuracy))


With evaluated results from BERT, We can proceed to using the GPT-3 model to compare the differences between the prompt-based hate speech detection and Encoder based.

We start with some preprocessing to remove the labels from the dataset.

In [ ]:
# Define the labels to be removed
labels_to_remove = ['L1: Type', 'L2: Abusiveness', 'L3: Target', 'L4: Demographic Characteristics', 'L5: Implicitness', 'L6: Profanity']

# Remove the rows with labels from the DataFrame
df = df[~df['Title'].isin(labels_to_remove)]

# Reset the index
df = df.reset_index(drop=True)

# Display the updated DataFrame
print(df.head()) # or df.info() to see the updated data

In [ ]:
import openai
import pandas as pd

# Set up OpenAI API credentials
openai.api_key = "SecretKey"

# Function to perform sentiment analysis using GPT-3
def analyze_sentiment_with_gpt3(text):
    # Specify the prompt for GPT-3
    prompt = f("Identify the features in the following text: '{text}' using GPT-3.\n\nL1: Type\t{{0,1,2,3}}\t0: Non-abusive, 1: Hate speech, 2: Derogatory, 3: Profanity\nL2: Abusiveness\t{{0,1}}\t0: Non-abusive, 1: Abusive\nL3: Target\t{{0,1}}\t0: Non-targeted, 1: Targeted\nL4: Demographic Characteristics\t{{0,1,2,3,4,5,6,7,8}}\t0: None, 1: Gender, 2: Sexual orientation, 3: Race, 4: Religion, 5: Disability, 6: Age, 7: Others, 8: Unclear\nL5: Implicitness\t{{0,1}}\t0: None, 1: Implicit (Containing implicit attacks.)\nL6: Profanity\t{{0,1}}\t0: None, 1: Profanity (Containing any words expressing abusiveness.)")


    # Generate text using GPT-3
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=2048,
        n=1,
        stop=None,
        temperature=0.5
    )

    # Extract the sentiment from the GPT-3 response
    sentiment = response["choices"][0]["text"].strip()

    return sentiment

# Loop through each row in the DataFrame and perform sentiment analysis using GPT-3
for index, row in df.iterrows():
    text = row['text']
    sentiment = analyze_sentiment_with_gpt3(text)
    df.at[index, 'sentiment'] = sentiment

# Print the updated DataFrame with sentiment analysis results
print(df)
